# Simple chain in LangChain powered by a device-to-Cloud model cascade

This tutorial shows how to define simple application logic in LangChain, use our
interop APIs to configure it to be powered by a cascade of models that spans
across a model in Cloud and an on-device model on Android, and deploy it in a
Java app on an Android phone. This illustrates many of the key interoperability
and portability benefits of GenC in one concise package. See the follow-up
tutorials listed in the parent directory for how you can further extend and
customize such logic to power more complex use cases.

## Initial setup

Before we begin, we need to setup your environment, such that you can continue
with the rest of this tutorial undisrupted.

*   First, you need to start a Jupyter notebook with the GenC dependency
    wired-in, and connect to that notebook - see [SETUP.md](../../../SETUP.md)
    at the root of the repo, and the supporting files in the
    [Jupyter setup directory](jupyter_setup/) for instructions how to setup the
    build and run environment and get Jupyter up and running.

*   Next, you need to setup access to the Gemini Pro model that will be used
    in the tutorials. Please see the
    [instructions](https://ai.google.dev/tutorials/rest_quickstart)
    on how to get an API key to access this model through Google AI Studio.

*   Finally, for the last portion of the tutorial that includes deployment on
    Android, you will need to download Gemma model for GPU (TODO link) and copy
    the .tflite model file to your Android phone. Make sure to set the same
    path in MODEL_PATH in steps below as the path you choose to copy the model
    file to.
    ```
    adb push {{src-model-dir}}/model_gpu.tflite /data/local/tmp/llm/model_gpu.tflite
    ```
    Please see instructions in (TODO link) to convert and download supported
    models on device.

TODO: external link for the on-device setup portion for the Gemma model

Now, to verify that GenC dependencies are loaded correctly, let's run a bunch
of imports we're going to use later.

In [ ]:
import generative_computing.python as genc
from generative_computing.python import authoring
from generative_computing.python import interop
from generative_computing.python import runtime
from generative_computing.python.examples import executor

## Defining application logic in LangChain

We're going to create here an example application logic using LangChain APIs.
For the sake of simplicy, let's go with a simple chain that consists of a prompt
template feeding into an LLM call. Let's define it as a function, so that we can
later play with different models.

In [ ]:
import langchain
from langchain.prompts import PromptTemplate

def create_my_chain(llm):
  return langchain.chains.LLMChain(
      llm=llm,
      prompt=PromptTemplate(
          input_variables=["topic"],
          template="Tell me about {topic}?",
      ),
  )

## Declaring a model you will use to power the chain

Now, let's define a model we can use. In GenC, we refer to models symbolically
since the same model may be provisioned differently depending on where you run
the code (recall that we want to demonstrate in this tutorial is running your
application logic in colab first, but then porting it to run on a phone).
To facilitate this, GenC provides interop APIs that enable you to declare the
use of a model, e.g., as shown below. For this tutorial, we're going to use
the Gemini Pro model from Google Studio AI.

Note: Please make sure you have an API_KEY to use as covered in the [Initial Setup section](#scrollTo=Cr8gU1bxnMjA&line=9&uniqifier=1) above.

In [ ]:
API_KEY = ""  #@param

my_cloud_model = genc.interop.langchain.CustomModel(
    uri="/cloud/gemini",
    config=genc.interop.gemini.create_config(API_KEY))

Now, you can construct the chain with it:

In [ ]:
my_chain = create_my_chain(my_cloud_model)

## Generating portable intermediate representation (IR)

Now that you have the application logic (the chain you defined above), we need
to translate it into what we call a *portable intermediate representation* (IR
for short) that can be deployed on an Android phone. You do this by calling the
converstion function provided by GenC, as follows:

In [ ]:
my_portable_ir = genc.interop.langchain.create_computation(my_chain)
print(my_portable_ir)

At the time of this writing, this converter only supports a subset of LangChain
functionality; we'll work to augment the coverage over time (and we welcome your
help if there's a feature of LangChain you'd like to see covered and are willing
to contribute it to the platform).

## Testing the IR locally in the Colab environment

Before we move over to deployment on Android, let's first test that the IR is
indeed working. While our goal is to run it on-device, we can just as well run
it here, in the colab environment (remember, all the code is portable). To do
this, we first need to construct a runtime instance:

In [ ]:
my_runtime = genc.examples.executor.create_default_executor()

Now, the constructor above is provided for convenience in running the examples
and tutorials, and is configured with a number of runtime capabilities that we
use in this context. Runtimes in GenC are fully modular and configurable, and
in most advanced uses, you'll want to configure a runtime that suits the
specific environment you want to run in, or your particular application (e.g.,
with additional custom dependencies, or without certain dependencies you don't
want in your environment). One of the tutorials later in the sequence explains
how to do that. For now, the default example runtime will suffice.

Given the runtime and the portable IR we want to run, we can construct a
*runner* object that will act like an ordinary Python function, and can
be directly invoked, like this:

In [ ]:
from generative_computing.python import runtime
my_runner = genc.runtime.Runner(my_portable_ir, my_runtime)

my_runner("scuba diving")

## Saving the IR to a file for deployment to phone

Now that you tested the IR locally, it's time to deploy it on your phone and
test it there. First, let's save the IR into a file on the local filesystem.

In [ ]:
with open("/tmp/genc_demo.pb", "wb") as f:
  f.write(my_portable_ir.SerializeToString())

## Install Generative Computing Demo app, deploy the IR file on the phone

1. First, let's build and install the Generative Computing Demo app.
  * TODO: Finalize Docker vs non-Docker setup and pre-built APK or APK build and install


2. Next, let's copy the IR file to Android phone. Run following command to copy
the file via ADB.
  * ```adb push /tmp/genc_demo.pb /data/local/tmp/genc_demo.pb```


## Run the demo on phone

Make sure your device is connected to internet.

1. Open the “Generative Computing Demo” app and type a topic in the UI. As a reminder, here is the prompt template we are using: "Tell me about {topic}?"
  * Example text to enter in UI: "scuba diving"

2. See the result. This is the result coming from the Cloud model (make sure
that the internet connection is available).

## Adding on-device model

Now, recall that what we promised to demonstrate in this tutorial running your
application logic on a phone, where it might be powered by an on-device LLM
while the the phone may be offline. To achieve this, we're going to need to
modify the IR to include the on-device model. First, let's declare the use of
an on-device model in LangChain, similarly to how we did above for the cloud
model.

NOTE: Make sure you have downloaded the on-device model on Android at MODEL_PATH as covered in the [initial setup section](#scrollTo=Cr8gU1bxnMjA&line=9&uniqifier=1) above.

In [ ]:
MODEL_PATH = "/data/local/tmp/llm/model_gpu.tflite"  #@param

my_on_device_model = genc.interop.langchain.CustomModel(
    uri="/device/llm_inference",
    config={"model_path": MODEL_PATH})

Now, we're going to combine the cloud and on-device models into a simple type
of moel cascade that spans across cloud and on-device LLMs. For simplicity's
sake, let's define a two-model cascade that first tries to hit a cloud backend
in case we're online, and that defaults to the use of an on-device model when offline.

In [ ]:
my_model_cascade = genc.interop.langchain.ModelCascade(models=[
    my_cloud_model, my_on_device_model])

my_chain = create_my_chain(my_model_cascade)

You could've chosen to order models in the cascade differently to achieve a
different behavior. Everything is customizable! In the next tutorial in the
sequence, we'll show you how you can construct an even more powerful routing
mechanism, where routing is based on query sensitivity. For now, this simple
cascade will suffice.

Now, all that we need to do is to re-generate the IR, once again save it to a
file, and load it on the phone as shown above.

In [ ]:
my_portable_ir = genc.interop.langchain.create_computation(my_chain)
with open("/tmp/genc_demo.pb", "wb") as f:
  f.write(my_portable_ir.SerializeToString())

Once you have the IR on the phone, make sure to re-run the demo app so that it
loads the updated IR. This time, play with setting the Airplane Mode on your
Android phone to On and Off, retry the query, and notice how the result changes
depending on the model in use (responses from the on-device and cloud models
tend to look differently). Under the hood, the cascade you defined prompts GenC
to first try the cloud model, but if it fails (while in airplane mode), it
simply falls back to the on-device model (if present).

TODO: Add a recorded demo link?